In [1]:
import pandas as pd
import numpy as np

from pandas_profiling import ProfileReport
from Datafun import column_cutter
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, cohen_kappa_score

C:\Users\damni\anaconda3\envs\learn-env\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Going forward, we will focus on the less is more philosophy and priortize the Crashes dataset for out model. Time permitting, we will attempt to incorporate the other datasets to further improve our modeling abilites. The main thing we want to determine are the primary contributing causes to the crashes that have been documented in hopes that we can indentify patterns and help improve overall safety

While we have removed some clutter from our Crashes dataset, there is still a fair amount of cleaning and maintence to perform before the dataset will make a worthwhile use for classification. The first thing we will focus on is out LN_COUNT column and see if we can create a classifier to help us more accurately identify the correct number of lanes based off of geographic location. While this may not be 100% accurate, I prefer this approach to improve upon out data rather than completely dropping this information. 

In our final model, I plan to separate out all of the data that does not have the primary cause identified. 

In [2]:
df = pd.read_csv('Crashes.csv')
df.head()

Unnamed: 0                                    CRASH_RECORD_ID     RD_NO  \
0           0  073682ef84ff827659552d4254ad1b98bfec24935cc9cc...  JB460108   
1           1  1560fb8a1e32b528fef8bfd677d2b3fc5ab37278b157fa...  JC325941   
2           2  009e9e67203442370272e1a13d6ee51a4155dac65e583d...  JA329216   
3           3  00e47f189660cd8ba1e85fc63061bf1d8465184393f134...  JC194776   
4           4  0126747fc9ffc0edc9a38abb83d80034f897db0f739eef...  JB200478   

               CRASH_DATE  POSTED_SPEED_LIMIT TRAFFIC_CONTROL_DEVICE  \
0  10/02/2018 06:30:00 PM                  10            NO CONTROLS   
1  06/27/2019 04:00:00 PM                  45            NO CONTROLS   
2  06/30/2017 04:00:00 PM                  35      STOP SIGN/FLASHER   
3  03/21/2019 10:50:00 PM                  30         TRAFFIC SIGNAL   
4  03/26/2018 02:23:00 PM                  35            NO CONTROLS   

       DEVICE_CONDITION WEATHER_CONDITION      LIGHTING_CONDITION  \
0           NO CONTROLS             CLEAR                DARKNESS   
1           NO CONTROLS             CLEAR                DAYLIGHT   
2  FUNCTIONING PROPERLY             CLEAR                DAYLIGHT   
3  FUNCTIONING PROPERLY             CLEAR  DARKNESS, LIGHTED ROAD   
4           NO CONTROLS             CLEAR                DAYLIGHT   

           FIRST_CRASH_TYPE  ... INJURIES_NON_INCAPACITATING  \
0      PARKED MOTOR VEHICLE  ...                         0.0   
1  SIDESWIPE SAME DIRECTION  ...                         0.0   
2                   TURNING  ...                         0.0   
3                   TURNING  ...                         0.0   
4      PARKED MOTOR VEHICLE  ...                         0.0   

   INJURIES_REPORTED_NOT_EVIDENT INJURIES_NO_INDICATION INJURIES_UNKNOWN  \
0                            0.0                    1.0              0.0   
1                            0.0                    2.0              0.0   
2                            0.0                    3.0              0.0   
3                            0.0                    2.0              0.0   
4                            0.0                    2.0              0.0   

  CRASH_HOUR CRASH_DAY_OF_WEEK CRASH_MONTH   LATITUDE  LONGITUDE  \
0         18                 3          10        NaN        NaN   
1         16                 5           6        NaN        NaN   
2         16                 6           6  41.741804 -87.740954   
3         22                 5           3  41.741804 -87.740954   
4         14                 2           3  41.953647 -87.732082   

                                   LOCATION  
0                                       NaN  
1                                       NaN  
2  POINT (-87.740953581987 41.741803598989)  
3  POINT (-87.740953581987 41.741803598989)  
4  POINT (-87.732081736006 41.953646899951)  

[5 rows x 42 columns]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416198 entries, 0 to 416197
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     416198 non-null  int64  
 1   CRASH_RECORD_ID                416198 non-null  object 
 2   RD_NO                          412429 non-null  object 
 3   CRASH_DATE                     416198 non-null  object 
 4   POSTED_SPEED_LIMIT             416198 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         416198 non-null  object 
 6   DEVICE_CONDITION               416198 non-null  object 
 7   WEATHER_CONDITION              416198 non-null  object 
 8   LIGHTING_CONDITION             416198 non-null  object 
 9   FIRST_CRASH_TYPE               416198 non-null  object 
 10  TRAFFICWAY_TYPE                416198 non-null  object 
 11  LANE_CNT                       198555 non-null  float64
 12  ALIGNMENT                     

Doing a quick search, we find that that largest road in Chicago has 14 lanes so we can safely assume any accidents with more than this must be a placeholder.

In [4]:
df.loc[df['LANE_CNT'] > 14] = np.nan
Crashes = df

In [5]:
Crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416198 entries, 0 to 416197
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     416013 non-null  float64
 1   CRASH_RECORD_ID                416013 non-null  object 
 2   RD_NO                          412244 non-null  object 
 3   CRASH_DATE                     416013 non-null  object 
 4   POSTED_SPEED_LIMIT             416013 non-null  float64
 5   TRAFFIC_CONTROL_DEVICE         416013 non-null  object 
 6   DEVICE_CONDITION               416013 non-null  object 
 7   WEATHER_CONDITION              416013 non-null  object 
 8   LIGHTING_CONDITION             416013 non-null  object 
 9   FIRST_CRASH_TYPE               416013 non-null  object 
 10  TRAFFICWAY_TYPE                416013 non-null  object 
 11  LANE_CNT                       198370 non-null  float64
 12  ALIGNMENT                     

In [6]:
Crashes['LANE_CNT'].value_counts()

2.0     90913
4.0     49492
1.0     32500
3.0      8644
0.0      8011
6.0      4494
5.0      1935
8.0      1906
7.0       184
10.0      161
9.0        65
11.0       30
12.0       29
14.0        5
13.0        1
Name: LANE_CNT, dtype: int64

In [7]:
Crashes_ln_model = Crashes[Crashes['LANE_CNT'].notnull()]

In [8]:
Crashes_ln_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198370 entries, 2 to 416126
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     198370 non-null  float64
 1   CRASH_RECORD_ID                198370 non-null  object 
 2   RD_NO                          198368 non-null  object 
 3   CRASH_DATE                     198370 non-null  object 
 4   POSTED_SPEED_LIMIT             198370 non-null  float64
 5   TRAFFIC_CONTROL_DEVICE         198370 non-null  object 
 6   DEVICE_CONDITION               198370 non-null  object 
 7   WEATHER_CONDITION              198370 non-null  object 
 8   LIGHTING_CONDITION             198370 non-null  object 
 9   FIRST_CRASH_TYPE               198370 non-null  object 
 10  TRAFFICWAY_TYPE                198370 non-null  object 
 11  LANE_CNT                       198370 non-null  float64
 12  ALIGNMENT                     

Now that we've filtered out the unusable lanes that we know don't exist and the rows that just don't have information for, we will then filter out the ones which we sadly don't have enough information for. The cutoff that we will make will be for lanes that we do not have at least 1000 observations for. Granted, this does pose the possibility of skewing our data which will be important to keep in mind later.

In [9]:
counts = Crashes_ln_model['LANE_CNT'].value_counts()
filtered_counts = counts[counts > 200].index

In [10]:
Crashes_ln_model = Crashes_ln_model[Crashes_ln_model['LANE_CNT'].isin(filtered_counts)]

In [11]:
Crashes_ln_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197895 entries, 2 to 416126
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     197895 non-null  float64
 1   CRASH_RECORD_ID                197895 non-null  object 
 2   RD_NO                          197893 non-null  object 
 3   CRASH_DATE                     197895 non-null  object 
 4   POSTED_SPEED_LIMIT             197895 non-null  float64
 5   TRAFFIC_CONTROL_DEVICE         197895 non-null  object 
 6   DEVICE_CONDITION               197895 non-null  object 
 7   WEATHER_CONDITION              197895 non-null  object 
 8   LIGHTING_CONDITION             197895 non-null  object 
 9   FIRST_CRASH_TYPE               197895 non-null  object 
 10  TRAFFICWAY_TYPE                197895 non-null  object 
 11  LANE_CNT                       197895 non-null  float64
 12  ALIGNMENT                     

In [12]:
Crashes_ln_model['LANE_CNT'].value_counts()

2.0    90913
4.0    49492
1.0    32500
3.0     8644
0.0     8011
6.0     4494
5.0     1935
8.0     1906
Name: LANE_CNT, dtype: int64

In [13]:
Crashes_ln_model = Crashes_ln_model[Crashes_ln_model['LATITUDE'].notnull()]
Crashes_ln_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197038 entries, 2 to 416126
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     197038 non-null  float64
 1   CRASH_RECORD_ID                197038 non-null  object 
 2   RD_NO                          197036 non-null  object 
 3   CRASH_DATE                     197038 non-null  object 
 4   POSTED_SPEED_LIMIT             197038 non-null  float64
 5   TRAFFIC_CONTROL_DEVICE         197038 non-null  object 
 6   DEVICE_CONDITION               197038 non-null  object 
 7   WEATHER_CONDITION              197038 non-null  object 
 8   LIGHTING_CONDITION             197038 non-null  object 
 9   FIRST_CRASH_TYPE               197038 non-null  object 
 10  TRAFFICWAY_TYPE                197038 non-null  object 
 11  LANE_CNT                       197038 non-null  float64
 12  ALIGNMENT                     

### We will now attempt to create a KNN model using only Latitude and Longitude

In [14]:
y = Crashes_ln_model['LANE_CNT']
X = Crashes_ln_model[['LATITUDE', 'LONGITUDE', 'STREET_NAME']]


In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197038 entries, 2 to 416126
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   LATITUDE     197038 non-null  float64
 1   LONGITUDE    197038 non-null  float64
 2   STREET_NAME  197038 non-null  object 
dtypes: float64(2), object(1)
memory usage: 6.0+ MB


In [15]:
X_dummies = pd.get_dummies(X)
X_dummies.shape

(197038, 1402)

1400 columns is a bit more than what I want to work with so we'll try to classify some of these streets into simpler identifiers such as AVEs, RDs, STs, and etc


In [16]:
X['STREET_NAME'].nunique()

1400

In [17]:
X['STREET_TYPE'] = X.apply(lambda x: x['STREET_NAME'].split()[-1], axis = 1)

C:\Users\damni\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
X.head()

LATITUDE  LONGITUDE    STREET_NAME STREET_TYPE
2   41.741804 -87.740954     CICERO AVE         AVE
3   41.741804 -87.740954     CICERO AVE         AVE
8   41.721290 -87.628510  WENTWORTH AVE         AVE
9   41.741804 -87.740954     CICERO AVE         AVE
11  41.794779 -87.623828  GARFIELD BLVD        BLVD

In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197038 entries, 2 to 416126
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   LATITUDE     197038 non-null  float64
 1   LONGITUDE    197038 non-null  float64
 2   STREET_NAME  197038 non-null  object 
 3   STREET_TYPE  197038 non-null  object 
dtypes: float64(2), object(2)
memory usage: 7.5+ MB


In [20]:
X = X.drop('STREET_NAME', axis=1)

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197038 entries, 2 to 416126
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   LATITUDE     197038 non-null  float64
 1   LONGITUDE    197038 non-null  float64
 2   STREET_TYPE  197038 non-null  object 
dtypes: float64(2), object(1)
memory usage: 6.0+ MB


In [22]:
X['STREET_TYPE'].value_counts()

AVE          98484
ST           61383
RD           12201
BLVD          7716
DR            7646
PL            1874
NB            1649
SB            1490
PKWY          1340
BROADWAY      1011
E              534
CT             337
W              294
HWY            239
O              163
TER             82
MARKET          57
PLAISANCE       50
M               50
IB              47
L               47
N               44
PLZ             36
WEST            28
J               27
WAY             26
H               21
OB              20
PARK            17
SD              16
DEARBORN        13
RAMP            13
G               11
B               11
F               10
C                8
OVERPASS         8
ACCESS           4
D                4
LN               4
K                3
BUSWAY           3
RL               2
S                2
XR               2
INNER            1
VIA              1
SALLE            1
GRENSHAW         1
STREET           1
DRIVE            1
SQ               1
BLVD.       

In [23]:
counts = X['STREET_TYPE'].value_counts()
filtered_counts = counts[counts < 450].index

In [24]:
X['STREET_TYPE'].loc[X['STREET_TYPE'].isin(filtered_counts)] = 'Other'

C:\Users\damni\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
X['STREET_TYPE'].value_counts()

AVE         98484
ST          61383
RD          12201
BLVD         7716
DR           7646
PL           1874
Other        1710
NB           1649
SB           1490
PKWY         1340
BROADWAY     1011
E             534
Name: STREET_TYPE, dtype: int64

In [26]:
X.head()

LATITUDE  LONGITUDE STREET_TYPE
2   41.741804 -87.740954         AVE
3   41.741804 -87.740954         AVE
8   41.721290 -87.628510         AVE
9   41.741804 -87.740954         AVE
11  41.794779 -87.623828        BLVD

In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197038 entries, 2 to 416126
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   LATITUDE     197038 non-null  float64
 1   LONGITUDE    197038 non-null  float64
 2   STREET_TYPE  197038 non-null  object 
dtypes: float64(2), object(1)
memory usage: 6.0+ MB


In [28]:
X_dummies = pd.get_dummies(X)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.25)

In [30]:
KNN = KNeighborsClassifier(n_neighbors=13)

In [31]:
KNN.fit(X_train, y_train)
test_preds = KNN.predict(X_test)

In [32]:
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds, average='macro')))
    print("Recall Score: {}".format(recall_score(labels, preds, average='macro')))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average='macro')))
    
print_metrics(y_test, test_preds)

Precision Score: 0.3642351181126012
Recall Score: 0.2811279645878683
Accuracy Score: 0.5548112058465287
F1 Score: 0.29547488615243667


In [41]:
def find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25):
    best_k = 0
    best_score = 0.0
    for k in range(min_k, max_k+1):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        preds = knn.predict(X_test)
        acc = accuracy_score(y_test, preds)
        if acc > best_score:
            best_k = k
            best_score = acc
    
    print("Best Value for k: {}".format(best_k))
    print("Accuracy-Score: {}".format(best_score))

In [42]:
find_best_k(X_train, y_train, X_test, y_test)

Best Value for k: 13
Accuracy-Score: 0.5583840844498579


Well, this isn't exactly promising just yet as the model is practically randomly picking values using only the lat and long values.

In [52]:
cohen_kappa_score(y_test, test_preds)

0.31786779005386534

In [33]:
svm = SVC(C = 150, class_weight='balanced', probability = True)

In [34]:
svm.fit(X_train, y_train)
test_preds = svm.predict(X_test)

In [35]:
print_metrics(y_test, test_preds)

Precision Score: 0.1744288776424705
Recall Score: 0.2553041756344613
Accuracy Score: 0.15968331303288671
F1 Score: 0.13016500122365554


C:\Users\damni\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
Crashes = df[df['PRIM_CONTRIBUTORY_CAUSE'] != ('UNABLE TO DETERMINE' or 'NOT APPLICABLE')]

In [ ]:
Crashes_exclude = df[df['PRIM_CONTRIBUTORY_CAUSE'] == ('UNABLE TO DETERMINE' or 'NOT APPLICABLE')]